<a href="https://colab.research.google.com/github/ArmaanSeth/RAG-OpenAI-Pinecone/blob/main/RAG_OpenAI_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing dependencied
!pip install langchain InstructorEmbedding tiktoken pinecone-client sentence-transformers openai unstructured "unstructured[all-docs]" poppler-utils pandoc langchain pandoc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Provide Your OPENAI API KEY")

Provide Your OPENAI API KEY··········


In [1]:
# using instructor embeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
instruct_embeddings=HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",model_kwargs={"device":"cuda"})

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [4]:
# Data Ingestion
import os
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Pinecone
os.environ["PINECONE_API_KEY"]="3fee5bcf-f151-4e19-82a6-22134a83d9d2"
os.environ["PINECONE_ENVIRON"]="gcp_starter"
def get_chunks():
  path="./data"
  docs=os.listdir(path)
  documents=[]
  for doc in docs:
    loader = UnstructuredFileLoader(os.path.join(path, doc))
    documents.append(loader.load()[0])
      # for page in pdf.pages:
      #     text+=page.extract_text()
  text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  chunks=text_splitter.split_documents(documents)
  chunks = [chunk.page_content for chunk in chunks]
  return chunks

chunks=get_chunks()
index_name="yardstick"
vectorstore=Pinecone.from_texts(chunks,embedding=instruct_embeddings,index_name=index_name)

In [10]:
# Loading vectorstore from Pinecone
vectorstore=Pinecone.from_existing_index(index_name,embedding=instruct_embeddings)

In [11]:
vectorstore.similarity_search("What is NLP?")

[Document(page_content='What is natural language processing? Natural language processing (NLP) refers to the branch of computer science—and more specifically, the branch of artificial intelligence or AI—concerned with giving computers the ability to understand text and spoken words in much the same way human beings can.\n\nNLP combines computational linguistics—rule-based modeling of human language—with statistical, machine learning, and deep learning models. Together, these technologies enable computers to process human language in the form of text or voice data and to ‘understand’ its full meaning, complete with the speaker or writer’s intent and sentiment.'),
 Document(page_content='NLP drives computer programs that translate text from one language to another, respond to spoken commands, and summarize large volumes of text rapidly—even in real time. There’s a good chance you’ve interacted with NLP in the form of voice-operated GPS systems, digital assistants, speech-to-text dictatio

In [ ]:
#Creating Conversation Chain
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationKGMemory
llm=ChatOpenAI()
memory = ConversationKGMemory(llm=llm,memory_key='chat_history', return_messages=True)
chain=ConversationalRetrievalChain.from_llm(llm=llm,retriever=vectorstore.as_retriever(),memory=memory,combine_docs_chain_kwargs={"prompt": prompt1},)

In [ ]:
chain({"question":"What is AI?"})

{'question': 'What is AI?',
 'chat_history': [],
 'answer': 'Artificial Intelligence (AI) is a broad field of science that encompasses not only computer science but also psychology, philosophy, linguistics, and other areas. AI is concerned with getting computers to do tasks that would normally require human intelligence. There are many definitions of AI, but some common characteristics include the ability to plan, reason, learn, sense, and build some kind of perception of knowledge and understanding. AI systems combine machine learning and other data analytics methods to achieve artificial intelligence capabilities.'}

In [ ]:
!pip install gradio==3.48.0 pydantic -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 33.3 MB/s eta 0:00:00


In [ ]:
#Using Gradio for chatbot interface
import gradio as gr

def chat(question,_):
  return f'{chain({"question":question})["answer"]}'

gr.ChatInterface(chat).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fcf552bf837f4e88a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
